# Partitioned Convolution

*This notebook uses [[Wefers, 2015]](README.md) as reference.*

Partitioned covolution is a technique that breaks up the input sequence and/or the filter impulse response into *blocks* to increase the efficency of convolution computations. 

## Input Partitioning

Input partitioning involves splitting up an input sequence of potentially indefinte length into blocks of length $B$. The following function below can be used to partition an input sequence of arbitrary size.

In [1]:
import numpy as np

def partition_input(input, B):
    """Partition input sequence of arbitrary size into blocks of length B. 
    The last block is zero padded if the input is not equally divisible by B.
    """
    partition_cnt = int(np.ceil(len(input) / B))
    zero_pad_cnt = partition_cnt * B - len(input)
    zero_padded_input = np.pad(input, (0, zero_pad_cnt))
    return np.reshape(zero_padded_input, (partition_cnt, B))

## Filter Partitioning

Filter partitioning involves splitting up a FIR filters impulse response into sections. There are several methods that can be used to partition a filters impulse response. The method that is selected defines the different classes of convolution algorithms that can be used.

Partitioning a filter with an impulse response of length-$N$ involves splitting up the impulse response into $P$ sub filters of length $N_0, ..., N_{P-1}$ such that: 

$$\sum_{i = 0}^{P-1} N_i = N$$

Or in cases where the subfilters are zero-padded, the following realtionship holds true:

$$\sum_{i = 0}^{P-1} N_i \ge N$$

Each sub filter impulse response $h_i(n)$ relates to a connected interval $n_i^{first} \le n \le n_i^{last}$ in the original filter impulse response $h(n)$, and all sub filters are adjoining, such that $n_{i+1}^{first} = n_i^{last} + 1$.

The *sub filter length* $N_i$ is defined as:
$$N_i = n_i^{last} - n_i^{first} + 1$$

The position of the sub filter within the original impulse response is:
$$n_i^{first} = \sum_{j<i} N_j$$

The original filter $h(n)$ can be described by the following composition of sub filters:

$$h(n) = \sum_{i = 0}^{P-1} h_i(n - n_i^{first})$$

Each consecutive sub filter impulse respose must be the same size or larger than the previous sub filter impulse response such that (TODO: discuss why this is...):

$$N_0 \le N_1 \le ... \le N_{P-1}$$

 If there is only 1 partition, the filter is called *unpartitioned*. If all the sub filters in a partition are of equal size, the filter is called a *uniform partition*, otherwise it is called a *non-uniform partition*.

## Filter Structure

Inserting the partitioned filter into the linear convolution equation gives us:

$$\begin{aligned}
y(n) \quad &= \quad x(n) * h(n) \\
&= \quad x(n) * \left[ \sum_{i=0}^{P-1} h_i(n - n_i^{first}) \right] 
\end{aligned}$$

And then converting the delays to time-shifted unit impulses gives us the following two equations: 

$$\begin{aligned}
y(n) \quad &= \quad \sum_{i=0}^{P-1} \left[ \left[ x(n) * \delta(n - n_i^{first}) \right] * h_i(n) \right] \\
&= \quad \sum_{i=0}^{P-1} \left[ \left[ x(n) * h_i(n) \right] * \delta(n - n_i^{first}) \right]
\end{aligned}$$

Both of these equations are equivalent since convolution is commutative. This illustrates that the delays can be moved around when performing the convolution.

## Classification of Partitioned Convolution Algorithms

The image below illustrates different classes of convolution algorithms based on the partitioning of the input sequence and the filter. Only *uniformly partitioned* input sequences are considered for real-time processing, although the other categories can be useful for offline processing.

![](images/partitioned_convolution_classification_R3.png 'Classes of Convolution Algorithms')